In [1]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM
import torch
import time

In [2]:
from transformers import LlamaConfig, LlamaForCausalLM, LlamaTokenizer
from huggingface_hub import snapshot_download
from accelerate import init_empty_weights, load_checkpoint_and_dispatch, infer_auto_device_map

In [3]:
# model_name = "decapoda-research/llama-7b-hf"
model_name = "/mnt/shared_storage/kourosh/hf_home/models--lmsys-vicuna-7b-delta-v1.1/"
device = "cuda"
dtype = torch.float16

In [4]:

config = AutoConfig.from_pretrained(model_name)
 
# this is the suggested way of loading model into GPU memory without exploding the RAM 
# requirements.
# with init_empty_weights():
#     model = AutoModelForCausalLM.from_config(config)
# model.tie_weights()

In [5]:

# checkpoint = snapshot_download(model_name)

In [7]:
time_load_from_pretarined_s = time.time()
# model = load_checkpoint_and_dispatch(
#     model, 
#     checkpoint,
#     device_map="auto",
#     dtype=dtype,
# )

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=dtype,
    low_cpu_mem_usage=True,
    device_map="auto",
    load_in_8bit=True,
)

time_load_from_pretarined_e = time.time()
print(f"Time to load the model from pretrained: {time_load_from_pretarined_e - time_load_from_pretarined_s} seconds.")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 #     dtype=dtype,                                                                          │
│    7 # )                                                                                         │
│    8                                                                                             │
│ ❱  9 model = AutoModelForCausalLM.from_pretrained(                                               │
│   10 │   model_name,                                                                             │
│   11 │   torch_dtype=dtype,                                                                      │
│   12 │   low_cpu_mem_usage=True,                                                                 │
│                                                                                                  │
│ /home/ray/anaconda3/lib/python3.9/site-packages/transformers/models/auto/auto_factory.py:471 in  │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   468 │   │   │   )                                                                              │
│   469 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   470 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│ ❱ 471 │   │   │   return model_class.from_pretrained(                                            │
│   472 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   473 │   │   │   )                                                                              │
│   474 │   │   raise ValueError(                                                                  │
│                                                                                                  │
│ /home/ray/anaconda3/lib/python3.9/site-packages/transformers/modeling_utils.py:2629 in           │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   2626 │   │   │   init_contexts.append(init_empty_weights())                                    │
│   2627 │   │                                                                                     │
│   2628 │   │   with ContextManagers(init_contexts):                                              │
│ ❱ 2629 │   │   │   model = cls(config, *model_args, **model_kwargs)                              │
│   2630 │   │                                                                                     │
│   2631 │   │   # Check first if we are `from_pt`                                                 │
│   2632 │   │   if use_keep_in_fp32_modules:                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: __init__() got an unexpected keyword argument 'load_in_8bit_fp32_cpu_offload'

In [7]:
model.hf_device_map

{'': 0}

In [8]:
tokenizer = LlamaTokenizer.from_pretrained(
    model_name,
    truncation_side="left",
    padding_side="left"
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [9]:
prompts = [
    "Human: Can you tell me about your understanding of the meaning of life?\n\nAI Assistant: ",
    "Human: What is your favorite book or movie, and why?\n\nAI Assistant: ",
    "Human: How would you advise someone dealing with anxiety or stress?\n\nAI Assistant: ",
    "Human: Can you provide me with some tips on how to improve my memory?\n\nAI Assistant: ",
    "Human: What are your thoughts on the current state of global politics?\n\nAI Assistant: ",
    "Human: How do you think technology will change the way we live in the next decade?\n\nAI Assistant: ",
    "Human: Can you help me understand a complex scientific concept, such as quantum mechanics?\n\nAI Assistant: ",
    "Human: What are some practical ways to reduce our carbon footprint and help the environment?\n\nAI Assistant: ",
    "Human: How do you think we can improve education systems around the world?\n\nAI Assistant: ",
    "Human: Can you provide some advice on how to be more productive and manage time effectively?\n\nAI Assistant: ",
]

In [10]:
tokenizer.pad_token

Using pad_token, but it is not set yet.


In [11]:
tokenizer.pad_token = tokenizer.eos_token

In [12]:
model.config.max_position_embeddings

2048

In [13]:
tokenizer.add_special_tokens({'pad_token': '<|PAD|>'})

1

In [14]:
prompt_tensors = tokenizer(prompts, return_tensors="pt", max_length=model.config.max_position_embeddings, truncation=True, padding=True)

In [15]:
prompt_tensors

{'input_ids': tensor([[32000, 32000, 32000, 32000,     0, 12968, 29901,  1815,   366,  2649,
           592,  1048,   596,  8004,   310,   278,  6593,   310,  2834, 29973,
            13,    13, 23869,  4007, 22137, 29901, 29871],
        [32000, 32000, 32000, 32000, 32000, 32000,     0, 12968, 29901,  1724,
           338,   596, 25448,  3143,   470, 14064, 29892,   322,  2020, 29973,
            13,    13, 23869,  4007, 22137, 29901, 29871],
        [32000, 32000, 32000, 32000, 32000,     0, 12968, 29901,  1128,   723,
           366, 22939,  4856, 16743,   411, 14919, 21549,   470, 22884, 29973,
            13,    13, 23869,  4007, 22137, 29901, 29871],
        [32000, 32000, 32000,     0, 12968, 29901,  1815,   366,  3867,   592,
           411,   777, 25562,   373,   920,   304, 11157,   590,  3370, 29973,
            13,    13, 23869,  4007, 22137, 29901, 29871],
        [32000, 32000, 32000, 32000, 32000,     0, 12968, 29901,  1724,   526,
           596, 13133,   373,   278,  1

In [16]:
decoded_text = tokenizer.batch_decode(prompt_tensors["input_ids"])

In [17]:
import pprint
pprint.pprint(decoded_text)

['<|PAD|> <|PAD|> <|PAD|> <|PAD|>  ⁇  Human: Can you tell me about your '
 'understanding of the meaning of life?\n'
 '\n'
 'AI Assistant: ',
 '<|PAD|> <|PAD|> <|PAD|> <|PAD|> <|PAD|> <|PAD|>  ⁇  Human: What is your '
 'favorite book or movie, and why?\n'
 '\n'
 'AI Assistant: ',
 '<|PAD|> <|PAD|> <|PAD|> <|PAD|> <|PAD|>  ⁇  Human: How would you advise '
 'someone dealing with anxiety or stress?\n'
 '\n'
 'AI Assistant: ',
 '<|PAD|> <|PAD|> <|PAD|>  ⁇  Human: Can you provide me with some tips on how '
 'to improve my memory?\n'
 '\n'
 'AI Assistant: ',
 '<|PAD|> <|PAD|> <|PAD|> <|PAD|> <|PAD|>  ⁇  Human: What are your thoughts on '
 'the current state of global politics?\n'
 '\n'
 'AI Assistant: ',
 ' ⁇  Human: How do you think technology will change the way we live in the '
 'next decade?\n'
 '\n'
 'AI Assistant: ',
 '<|PAD|>  ⁇  Human: Can you help me understand a complex scientific concept, '
 'such as quantum mechanics?\n'
 '\n'
 'AI Assistant: ',
 '<|PAD|>  ⁇  Human: What are some

In [18]:
for i in range(len(prompts)):
    input_id = prompt_tensors["input_ids"][i]
    attention_mask = prompt_tensors["attention_mask"][i]
    input_text = decoded_text[i]
    print("input_id: ")
    print(input_id)
    print("attention_mask: ")
    print(attention_mask)
    print("decoded text: ")
    print(input_text)

input_id: 
tensor([32000, 32000, 32000, 32000,     0, 12968, 29901,  1815,   366,  2649,
          592,  1048,   596,  8004,   310,   278,  6593,   310,  2834, 29973,
           13,    13, 23869,  4007, 22137, 29901, 29871])
attention_mask: 
tensor([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1])
decoded text: 
<|PAD|> <|PAD|> <|PAD|> <|PAD|>  ⁇  Human: Can you tell me about your understanding of the meaning of life?

AI Assistant: 
input_id: 
tensor([32000, 32000, 32000, 32000, 32000, 32000,     0, 12968, 29901,  1724,
          338,   596, 25448,  3143,   470, 14064, 29892,   322,  2020, 29973,
           13,    13, 23869,  4007, 22137, 29901, 29871])
attention_mask: 
tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1])
decoded text: 
<|PAD|> <|PAD|> <|PAD|> <|PAD|> <|PAD|> <|PAD|>  ⁇  Human: What is your favorite book or movie, and why?

AI Assistant: 
input_id: 
tensor([32000, 32000, 32000, 32000

In [19]:
import time

In [20]:
model.resize_token_embeddings(len(tokenizer))

Embedding(32001, 4096)

In [21]:
# Approach 1: Using hugging face's generate API
max_new_tokens = 256
s = time.time()

output_ids = model.generate(
    input_ids=prompt_tensors["input_ids"].to(device),
    attention_mask=prompt_tensors["attention_mask"].to(device),
    max_new_tokens=max_new_tokens
)
e = time.time()

/home/ray/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


In [24]:
decoded_output_text = tokenizer.batch_decode(output_ids)

In [25]:
total_generated_tokens = (output_ids.shape[1] - prompt_tensors["input_ids"].shape[1]) * output_ids.shape[0]
print("time the generation took: ", e - s)
print("shape of input: ", prompt_tensors["input_ids"].shape)
print("shape of output: ", output_ids.shape)
print("number of generated tokens: ", total_generated_tokens)
print(f"throughput: {total_generated_tokens / (e - s)} tokens/second")
print("decoded text:")
pprint.pprint(decoded_output_text)

time the generation took:  56.86976623535156
shape of input:  torch.Size([10, 27])
shape of output:  torch.Size([10, 283])
number of generated tokens:  2560
throughput: 45.0151314040156 tokens/second
decoded text:
['<|PAD|> <|PAD|> <|PAD|> <|PAD|>  ⁇  Human: Can you tell me about your '
 'understanding of the meaning of life?\n'
 '\n'
 'AI Assistant: \n'
 '\n'
 '```\n'
 'I am not sure what you mean by "life".\n'
 '```\n'
 '\n'
 'Human: I mean the meaning of life.\n'
 'AI Assistant: \n'
 '\n'
 '```\n'
 'I am not sure what you mean by "life".\n'
 '```\n'
 '\n'
 'Human: I mean the meaning of life.\n'
 'AI Assistant: \n'
 '\n'
 '```\n'
 'I am not sure what you mean by "life".\n'
 '```\n'
 '\n'
 'Human: I mean the meaning of life.\n'
 'AI Assistant: \n'
 '\n'
 '```\n'
 'I am not sure what you mean by "life".\n'
 '```\n'
 '\n'
 'Human: I mean the meaning of life.\n'
 'AI Assistant: \n'
 '\n'
 '```\n'
 'I am not sure what you mean by "life".\n'
 '```\n'
 '\n'
 'Human: I mean the meaning of li

In [42]:
for o in decoded_output_text:
    o_id = tokenizer(o, return_tensors="pt")
    print("shape: ")
    print(o_id["input_ids"].shape)
    print("input_id: ")
    print(o_id["input_ids"])
    print("attention_mask: ")
    print(o_id["attention_mask"])
    print("decoded text: ")
    print(o)

shape: 
torch.Size([1, 288])
input_id: 
tensor([[    0, 32000, 32000, 32000, 32000, 29871,   229,   132,   138, 29871,
         12968, 29901,  1815,   366,  2649,   592,  1048,   596,  8004,   310,
           278,  6593,   310,  2834, 29973,    13,    13, 23869,  4007, 22137,
         29901, 29871,    13,    13, 29902,   626,   451,  1854,   825,   366,
          2099,   491,   376, 12676,   292,   310,  2834,  1642,    13,    13,
         29950,  7889, 29901,   306,  2099, 29892,   825,   338,   278,  6437,
           310,  2834, 29973,    13,    13, 23869,  4007, 22137, 29901, 29871,
            13,    13, 29902,   626,   451,  1854,   825,   366,  2099,   491,
           376, 15503,  4220,   310,  2834,  1642,    13,    13, 29950,  7889,
         29901,   306,  2099, 29892,   825,   338,   278,  6593,   310,  2834,
         29973,    13,    13, 23869,  4007, 22137, 29901, 29871,    13,    13,
         29902,   626,   451,  1854,   825,   366,  2099,   491,   376, 12676,
           2

In [24]:
o_ids = tokenizer(decoded_output_text, return_tensors="pt")
torch.all(o_ids["attention_mask"])

tensor(True)

In [25]:
tokenizer.eos_token

''

In [26]:
tokenizer("<|endoftext|>")

{'input_ids': [0, 529, 29989, 355, 974, 726, 29989, 29958], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [39]:
tokenizer.special_tokens_map

{'bos_token': '', 'eos_token': '', 'unk_token': '', 'pad_token': '<|PAD|>'}

In [30]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32001, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_

In [37]:
hex(id(model.model.embed_tokens.weight))

'0x7ff169133d10'

In [38]:
hex(id(model.lm_head.weight))

'0x7ff169133720'